<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Fine_Tuning_BERT_for_Text_Classification_Knowledge_Distillation_Quantization_17_01_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets  -q

In [5]:
import pandas as pd
from datasets import DatasetDict, Dataset

In [7]:
# data downloaded from here: https://www.kaggle.com/datasets/taruntiwarihp/phishing-site-urls/data
df = pd.read_csv("/content/phishing_site_urls.csv")

In [8]:
df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


#Data Preparation

In [9]:
# drop data
df = df.dropna()

# create dataframes from each class

df_safe = df[df['Label']=="good"]
df_not_safe = df[df['Label']=="bad"]


In [10]:
# define number of samples to keep
num_samples = 150

# Sample min_size rows from each class to ensure a 50-50 split
df_safe_sample = df_safe.sample(num_samples, random_state=42)
df_not_safe_sample = df_not_safe.sample(num_samples, random_state=42)

In [11]:
df_safe_sample.head()

,URL,Label
313746,depositaccounts.com/savings/,good
305419,citypages.com/related/to/Dave+Simonett/,good
284414,askart.com/askart/c/kate_carew/kate_carew.aspx,good
153114,brianwattsphoto.com/,good
444475,thefreedictionary.com/action+deferred,good


In [12]:
# replace "Email Type" with Boolean flag "isPhising"
df_safe_sample = df_safe_sample.assign(isPhishing=False)
df_safe_sample = df_safe_sample.drop('Label',axis=1)


df_not_safe_sample = df_not_safe_sample.assign(isPhishing=True)
df_not_safe_sample = df_not_safe_sample.drop('Label',axis=1)

In [13]:
df_safe_sample.head()

,URL,isPhishing
313746,depositaccounts.com/savings/,False
305419,citypages.com/related/to/Dave+Simonett/,False
284414,askart.com/askart/c/kate_carew/kate_carew.aspx,False
153114,brianwattsphoto.com/,False
444475,thefreedictionary.com/action+deferred,False


In [14]:
balanced_df = pd.concat([df_safe_sample, df_not_safe_sample])
balanced_df.columns = ['text', 'labels']

# convert labels column to int
balanced_df['labels'] = balanced_df['labels'].astype(int)

# Shuffle the balanced dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_df.head()

,text,labels
0,account.buscaunamascota.interactivos123.com/co...,1
1,sabiliku.com/boss2/,1
2,www.nembi.com.br/plugins/user/paypal/login.php...,1
3,absoluteastronomy.com/topics/Casimir_Pierre_Pe...,0
4,cnajs.com/pic/Agatha%20Christie/dropbox/dropbox/,1


In [15]:
balanced_df.shape

(300, 2)

In [17]:
train_frac = 0.7
valid_frac = 0.15
test_frac = 0.15

# define train and validation size
train_size = int(train_frac * len(balanced_df))
valid_size = int(valid_frac * len(balanced_df))

# create train, validation, and test datasets
train_df = balanced_df[:train_size]
valid_df = balanced_df[train_size:train_size + valid_size]
test_df = balanced_df[train_size + valid_size:]

# Convert the pandas DataFrames back to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)

In [18]:
# Combine into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_ds,
    'validation': valid_ds,
    'test': test_ds
})

In [19]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 45
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 45
    })
})

In [21]:
# push data to hub
# dataset_dict.push_to_hub("shawhin/phishing-site-classification")

# Teacher model training

In [25]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.7 MB/s eta 0:00:00


In [27]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [30]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
#Freeze Base Model
for name, param in model.named_parameters():
  print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [40]:
# freeze base model parameters
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
  if "pooler" in name:
    param.requires_grad = True

#Preprocess Text

In [45]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [46]:
tokenized_data = dataset_dict.map(preprocess_function, batched = True)

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [47]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [52]:
#load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")


def compute_metrics(eval_pred):
  # get predictions
  predictions , labels = eval_pred

  # apply softmax to get probabilities
  probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims = True)

  # use probabilities of the positive class for ROC AUC
  positive_class_probs = probabilities[:,1]

  #compute auc
  auc = np.round(auc_score.compute(prediction_scores = positive_class_probs, references = labels)["roc_auc"], 3)

  # predict most probable class
  predicted_classes = np.argmax(predictions, axis = 1)

  # compute accuracy
  acc = np.round(accuracy.compute(predictions = predicted_classes, references = labels)["accuracy"], 3)

  return {"Accuracy": acc, "AUC": auc}

In [49]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

In [50]:
training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-53-25805d4bc91c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.558700,0.503816,0.778000,0.937000
2,0.511700,0.454399,0.844000,0.931000
3,0.487300,0.415091,0.778000,0.929000
4,0.444300,0.458749,0.756000,0.935000
5,0.481500,0.380805,0.800000,0.939000
6,0.404300,0.389158,0.756000,0.941000
7,0.401400,0.435890,0.756000,0.935000
8,0.417000,0.403762,0.756000,0.933000
9,0.374000,0.399394,0.756000,0.933000
10,0.340700,0.391540,0.756000,0.933000


TrainOutput(global_step=270, training_loss=0.44207338050559714, metrics={'train_runtime': 876.831, 'train_samples_per_second': 2.395, 'train_steps_per_second': 0.308, 'total_flos': 77022678123960.0, 'train_loss': 0.44207338050559714, 'epoch': 10.0})

In [55]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.889, 'AUC': 0.923}


In [56]:
# push model to hub
# trainer.push_to_hub()

# Knowledge Distillation

In [57]:
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DistilBertForSequenceClassification, DistilBertConfig

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [59]:
# data = load_dataset("shawhin/phishing-site-classification")
# data

In [58]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 45
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 45
    })
})

In [67]:
# device = torch.device('cuda')

In [68]:
# Load teacher model and tokenizer
model_path = "/content/bert-phishing-classifier_teacher/checkpoint-108"

tokenizer = AutoTokenizer.from_pretrained(model_path)
# teacher_model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
teacher_model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [69]:
# Load student model
my_config = DistilBertConfig(n_heads=4, n_layers=2) # drop 8 heads per layer and 4 layers (originally it has 12 n_heads and 6 n_layers)

# student_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
#                                                                     config=my_config,).to(device)


student_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                                    config=my_config,)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Tokenization

In [71]:
# define text preprocessing
def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

# tokenize all datasetse
tokenized_data = dataset_dict.map(preprocess_function, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [72]:
# evaluation function


# Function to evaluate model performance
# def evaluate_model(model, dataloader, device):
def evaluate_model(model, dataloader):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    # Disable gradient calculations
    with torch.no_grad():
        for batch in dataloader:
            # input_ids = batch['input_ids'].to(device)
            # attention_mask = batch['attention_mask'].to(device)
            # labels = batch['labels'].to(device)
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            # Forward pass to get logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions
            # preds = torch.argmax(logits, dim=1).cpu().numpy()
            preds = torch.argmax(logits, dim=1).numpy()
            all_preds.extend(preds)
            # all_labels.extend(labels.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

# Train student model

In [73]:
# Function to compute distillation and hard-label loss
def distillation_loss(student_logits, teacher_logits, true_labels, temperature, alpha):
    # Compute soft targets from teacher logits
    soft_targets = nn.functional.softmax(teacher_logits / temperature, dim=1)
    student_soft = nn.functional.log_softmax(student_logits / temperature, dim=1)

    # KL Divergence loss for distillation
    distill_loss = nn.functional.kl_div(student_soft, soft_targets, reduction='batchmean') * (temperature ** 2)

    # Cross-entropy loss for hard labels
    hard_loss = nn.CrossEntropyLoss()(student_logits, true_labels)

    # Combine losses
    loss = alpha * distill_loss + (1.0 - alpha) * hard_loss

    return loss

In [74]:
# hyperparameters
batch_size = 32
lr = 1e-4
num_epochs = 5
temperature = 2.0
alpha = 0.5

# define optimizer
optimizer = optim.Adam(student_model.parameters(), lr=lr)

# create training data loader
dataloader = DataLoader(tokenized_data['train'], batch_size=batch_size)
# create testing data loader
test_dataloader = DataLoader(tokenized_data['test'], batch_size=batch_size)

In [75]:
# put student model in train mode
student_model.train()

# train model
for epoch in range(num_epochs):
    for batch in dataloader:
        # Prepare inputs
        # input_ids = batch['input_ids'].to(device)
        # attention_mask = batch['attention_mask'].to(device)
        # labels = batch['labels'].to(device)

        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Disable gradient calculation for teacher model
        with torch.no_grad():
            teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_outputs.logits

        # Forward pass through the student model
        student_outputs = student_model(input_ids, attention_mask=attention_mask)
        student_logits = student_outputs.logits

        # Compute the distillation loss
        loss = distillation_loss(student_logits, teacher_logits, labels, temperature, alpha)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

    # Evaluate the teacher model
    # teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, test_dataloader, device)
    teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, test_dataloader)
    print(f"Teacher (test) - Accuracy: {teacher_accuracy:.4f}, Precision: {teacher_precision:.4f}, Recall: {teacher_recall:.4f}, F1 Score: {teacher_f1:.4f}")

    # Evaluate the student model
    # student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, test_dataloader, device)
    student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, test_dataloader)
    print(f"Student (test) - Accuracy: {student_accuracy:.4f}, Precision: {student_precision:.4f}, Recall: {student_recall:.4f}, F1 Score: {student_f1:.4f}")
    print("\n")

    # put student model back into train mode
    student_model.train()

Epoch 1 completed with loss: 0.5535222887992859
Teacher (test) - Accuracy: 0.7556, Precision: 1.0000, Recall: 0.5000, F1 Score: 0.6667
Student (test) - Accuracy: 0.5333, Precision: 1.0000, Recall: 0.0455, F1 Score: 0.0870


Epoch 2 completed with loss: 0.475430965423584
Teacher (test) - Accuracy: 0.7556, Precision: 1.0000, Recall: 0.5000, F1 Score: 0.6667
Student (test) - Accuracy: 0.6444, Precision: 0.7143, Recall: 0.4545, F1 Score: 0.5556


Epoch 3 completed with loss: 0.3283788561820984
Teacher (test) - Accuracy: 0.7556, Precision: 1.0000, Recall: 0.5000, F1 Score: 0.6667
Student (test) - Accuracy: 0.8444, Precision: 0.8261, Recall: 0.8636, F1 Score: 0.8444


Epoch 4 completed with loss: 0.26990431547164917
Teacher (test) - Accuracy: 0.7556, Precision: 1.0000, Recall: 0.5000, F1 Score: 0.6667
Student (test) - Accuracy: 0.7556, Precision: 0.7895, Recall: 0.6818, F1 Score: 0.7317


Epoch 5 completed with loss: 0.22831597924232483
Teacher (test) - Accuracy: 0.7556, Precision: 1.0000, R

# Evaluate Models

In [77]:
# create testing data loader
validation_dataloader = DataLoader(tokenized_data['validation'], batch_size=8)

# Evaluate the teacher model
# teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, validation_dataloader, device)
teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, validation_dataloader)
print(f"Teacher (validation) - Accuracy: {teacher_accuracy:.4f}, Precision: {teacher_precision:.4f}, Recall: {teacher_recall:.4f}, F1 Score: {teacher_f1:.4f}")

# Evaluate the student model
# student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, validation_dataloader, device)
student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, validation_dataloader)
print(f"Student (validation) - Accuracy: {student_accuracy:.4f}, Precision: {student_precision:.4f}, Recall: {student_recall:.4f}, F1 Score: {student_f1:.4f}")

Teacher (validation) - Accuracy: 0.7333, Precision: 0.9231, Recall: 0.5217, F1 Score: 0.6667
Student (validation) - Accuracy: 0.7333, Precision: 1.0000, Recall: 0.4783, F1 Score: 0.6471


# Push to hub

In [78]:
# from huggingface_hub import notebook_login

# notebook_login()

In [79]:
# student_model.push_to_hub("bert-phishing-classifier_student")

# save the student model

In [80]:
# Define the path where you want to save the model
save_directory = "./student model"

# Save the model
student_model.save_pretrained(save_directory)

# If you also want to save the tokenizer, you can do this:
# tokenizer.save_pretrained(save_directory)

# Quantization

In [101]:
!pip install datasets -q
!pip install -U bitsandbytes transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BitsAndBytesConfig

In [84]:
# data = load_dataset("phishing-site-classification")
# data

In [3]:
# Load student model and techer tokenizer from hub directory

# model_id = "bert-phishing-classifier_student"

# Define the directory where the model is saved
load_directory = "./student model"

# Load the model and tokenizer from the local directory
model_id = AutoModelForSequenceClassification.from_pretrained(load_directory)


# tokenizer = AutoTokenizer.from_pretrained("bert-phishing-classifier_teacher")
model_path = "/content/bert-phishing-classifier_teacher/checkpoint-108"
tokenizer = AutoTokenizer.from_pretrained(model_path)


# model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)
# model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [4]:
# load model in model as 4-bit
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# model_nf4 = AutoModelForSequenceClassification.from_pretrained(model_id, device_map=device, quantization_config=nf4_config)

model_nf4 = AutoModelForSequenceClassification.from_pretrained(load_directory,quantization_config=nf4_config)

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [76]:
# Evaluate post quantization
# Evaluate the student model
# quantized_accuracy, quantized_precision, quantized_recall, quantized_f1 = evaluate_model(model_nf4, validation_dataloader, device)
quantized_accuracy, quantized_precision, quantized_recall, quantized_f1 = evaluate_model(model_nf4, validation_dataloader)

print("Post-quantization Performance")
print(f"Accuracy: {quantized_accuracy:.4f}, Precision: {quantized_precision:.4f}, Recall: {quantized_recall:.4f}, F1 Score: {quantized_f1:.4f}")

In [5]:
# from huggingface_hub import notebook_login

# notebook_login()

In [6]:
# model_nf4.push_to_hub("bert-phishing-classifier_student_4bit")